In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
%ls '/kaggle/input/gtsrb-german-traffic-sign/'

Meta/  Meta.csv  Test/  Test.csv  Train/  Train.csv  meta/  test/  train/


In [2]:
print(pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv').head())

   Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0     27      26       5       5      22      20       20   
1     28      27       5       6      23      22       20   
2     29      26       6       5      24      21       20   
3     28      27       5       6      23      22       20   
4     28      26       5       5      23      21       20   

                             Path  
0  Train/20/00020_00000_00000.png  
1  Train/20/00020_00000_00001.png  
2  Train/20/00020_00000_00002.png  
3  Train/20/00020_00000_00003.png  
4  Train/20/00020_00000_00004.png  


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import torch

print(torch.cuda.is_available()) 

train_trans = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Resize((128,128))
])

data_train = ImageFolder(
    '/kaggle/input/gtsrb-german-traffic-sign/Train',
    transform= train_trans
)

dataloader_train = DataLoader(
    data_train,
    shuffle = True,
    batch_size = 1000
)

#change dataloader batch size to 1 before running 
# image, label = next(iter(dataloader_train))
# print(image.shape)
# image = image.squeeze().permute(1,2,0)
# print(image.shape)
# plt.imshow(image)
# plt.show()

True


In [4]:
import torch.nn as nn
class TrafficSign(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding =1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.25),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*32*32, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x= self.classifier(x)
        return x 
    
    

In [5]:
ts = TrafficSign(num_classes = 43).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ts.parameters(), lr=1e-3)

for epoch in range(100):
    running_loss = 0.0
    
    for images, labels in dataloader_train:
        images = images.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = ts(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss/len(dataloader_train)
    print(f'Epoch {epoch + 1} loss is {epoch_loss:.4f}')

Epoch 1 loss is 3.3091
Epoch 2 loss is 2.3014
Epoch 3 loss is 1.7238
Epoch 4 loss is 1.4345
Epoch 5 loss is 1.2713
Epoch 6 loss is 1.1471
Epoch 7 loss is 1.0538
Epoch 8 loss is 0.9752
Epoch 9 loss is 0.9229
Epoch 10 loss is 0.8717
Epoch 11 loss is 0.8441
Epoch 12 loss is 0.7929
Epoch 13 loss is 0.7513
Epoch 14 loss is 0.7148
Epoch 15 loss is 0.6999
Epoch 16 loss is 0.6868
Epoch 17 loss is 0.6659
Epoch 18 loss is 0.6396
Epoch 19 loss is 0.6230
Epoch 20 loss is 0.6129
Epoch 21 loss is 0.5922
Epoch 22 loss is 0.5851
Epoch 23 loss is 0.5682
Epoch 24 loss is 0.5595
Epoch 25 loss is 0.5441
Epoch 26 loss is 0.5392
Epoch 27 loss is 0.5211
Epoch 28 loss is 0.5229
Epoch 29 loss is 0.5280
Epoch 30 loss is 0.5051
Epoch 31 loss is 0.4845
Epoch 32 loss is 0.4870
Epoch 33 loss is 0.4761
Epoch 34 loss is 0.4787
Epoch 35 loss is 0.4701
Epoch 36 loss is 0.4691
Epoch 37 loss is 0.4586
Epoch 38 loss is 0.4602
Epoch 39 loss is 0.4597
Epoch 40 loss is 0.4448
Epoch 41 loss is 0.4354
Epoch 42 loss is 0.4309
E

In [6]:
print(pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv').head())

   Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId            Path
0     53      54       6       5      48      49       16  Test/00000.png
1     42      45       5       5      36      40        1  Test/00001.png
2     48      52       6       6      43      47       38  Test/00002.png
3     27      29       5       5      22      24       33  Test/00003.png
4     60      57       5       5      55      52       11  Test/00004.png


In [7]:
from PIL import Image

class TestDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['Path']  
        img_path = os.path.join(self.img_dir, img_name) # NB drop the test in the img_dir because of how path is inputted
        image = Image.open(img_path).convert('RGB')
        
        label = self.data.iloc[idx]['ClassId']  
        
        if self.transform:
            image = self.transform(image)
        
        return image, label 

In [8]:
test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128))
])

dataset_test = TestDataset(
    csv_file = '/kaggle/input/gtsrb-german-traffic-sign/Test.csv',
    img_dir='/kaggle/input/gtsrb-german-traffic-sign/',
    transform= test_trans
)

dataloader_test = DataLoader(
    dataset_test,
    shuffle = True,
    batch_size = 1500
)

#change dataloader batch size to 1 before running 
image, label = next(iter(dataloader_test))
# print(image.shape)
# image = image.squeeze().permute(1,2,0)
# print(image.shape)
# plt.imshow(image)
# plt.show()
print(label)

tensor([ 4, 13, 10,  ...,  8, 10,  3])


In [9]:
from torchmetrics import Accuracy, Precision, Recall
recall = Recall(task='multiclass', num_classes=43, average='micro').to('cuda')
accuracy = Accuracy(task='multiclass', num_classes=43, average='micro').to('cuda')
precision = Precision(task='multiclass', num_classes=43, average='micro').to('cuda')

ts.eval()

with torch.no_grad():
    for images, labels in dataloader_test:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = ts(images)
        _, preds = torch.max(outputs,1)
        precision(preds, labels)
        accuracy(preds, labels)
        recall(preds, labels)

ts_precision = precision.compute()
ts_accuracy = accuracy.compute()
ts_recall = recall.compute()

print(f'Precision:{ts_precision}\nAccuracy:{ts_accuracy}\nRecall:{ts_recall}')

Precision:0.05859065800905228
Accuracy:0.05859065800905228
Recall:0.05859065800905228
